Write a program will except input to select a chosen data set and a JSON-encoded search criteria to select records from the chosen data set.  The write the matches from the search to a JSON file.

In [1]:

import json
import csv
#import pandas as pd
from pprint import pprint as pp
import datetime 


In [2]:
def log_it(row_count,row_found,main_key):
    '''
    build an entry in the log file dictionary
    '''
    log_entry = {}
    log_detail = {}
    #datetimeObject = datetime.datetime.today()
    dt = datetime.datetime.now()
    dt_string = dt.strftime("Date: %m/%d/%Y  time: %H:%M:%S")
    dt_key = dt.strftime("%m/%d/%Y-%H:%M:%S")
    #log_time = print(datetimeObject.time())
    
    log_detail.update({'search_name': main_key, 'records_read': row_count, 'record_returned' : row_found,
                      'log_date_time':dt_string}) 
    log_entry[dt_key]= log_detail
    #pp(log_entry)
    return log_entry
     

In [3]:
def correct_value(dictionary_value, search_value, search_action):
    '''
    Determine if the search value matches the dictionary value based on the operand
    inputs - dictionary value for search_field key, the search value for the search_value key and the search operand 
    returns Boolean True if the search value matches the dictionary value based on the operand
    returns Boolean False if the search value does not match the dictionary value based on the operand 
    '''
    
    if search_action == '=':
        if dictionary_value == search_value:
            return True
        else:
            return False
        
    if search_action == '>=':
        if dictionary_value >= search_value:
            return True
        else:
            return False
        
    if search_action == '<=':
        if dictionary_value <= search_value:
            return True
        else:
            return False
     
    if search_action == '<':
        if dictionary_value < search_value:
            return True
        else:
            return False 
        
    if search_action == '>':
        if dictionary_value > search_value:
            return True
        else:
            return False    

In [4]:
# prompt the user for dataset path name and read file in order to determine it is a valid path name
def path_names(file_type):
    '''
    prompt the user for chosen dataset path name and read file in order to determine it is a valid path name
    input parameter is 'data' to prompt for name of data file otherwise will prompt for search file name
    returns the name of the validiated file name    
    '''
    not_validiated = True
    while not_validiated:
        if file_type == 'data':
            path_name_s = (input('Enter a path for the input file: \n'))
        else:
            path_name_s = (input('Enter a path for the search file: \n'))    
        if len(path_name_s) > 0:
            try:
                file = open(path_name_s, 'r')
                line = file.read()
                open_file = True
                file.close()
            except:
                print("I/O Error on read of file, please try again")
                open_file = False
            finally:
                #print('open_file', open_file)
                if open_file:
                    not_validiated = False 
    return path_name_s

In [5]:
def build_search_criteria(search_file_path):
    '''
    reads the json search file and begins the search of the data file
    input - file path name for search file
    returns - search criteria from the json file as a dictionary    
    '''
    with open(search_file_path) as json_data:
        sc_dict = json.load(json_data)
        return sc_dict

In [6]:
def search_csv(search_field, search_value, search_action, csv_data_file):
    '''
    reads the CSV dictionary with input search values
    input - search_field, search_value, search_action
    returns - results of the search in a json file    
    '''
    #print('search_csv',search_field, search_value, search_action)
    
    #initialize dicts
    return_dict = {}
    row_count = 0
    row_found = 0
    row_id = 0
    for row in csv_data_file:
        row_count += 1
        dictionary_value = row.get(search_field,'Null')
        #print('found_field', found_field)
        if dictionary_value == 'Null':
            print(f'An error occurred; {search_field} was not found in the input file')
        else:
            #search_field was found, now call routine to determine if correct value
            fould_value = False
            fould_value = correct_value(dictionary_value, search_value, search_action)
            #print('fould_value', fould_value)
            if fould_value == True:
                row_found += 1
                row_id += 1
                return_dict[row_id] = row
    #return the results        
    return return_dict, row_count, row_found       

    
    

In [7]:
def load_csv(input_file_path):
    '''
    reads the json search file and begins the search of the data file
    input - file path name for search file
    returns - results of the search in a json file    
    '''
    
    #load csv file to dictionary
     
    file = open(input_file_path, newline='')
    reader = csv.DictReader(file)
    #print(reader)
    
    return reader
          

In [8]:
def process_search(search_file_path, input_file_path):
    '''
    reads the json search file and begins the search of the data file
    inputs - file path name for search file and the input file
    returns - results of the search in a json file    
    '''
    #build a dictionary that contains the search criteria from the JSON file
    search_criteria_dict = build_search_criteria(search_file_path)
    
    keys = search_criteria_dict.keys() 
    #get search values by looping thru search dictionary
    for main_key in keys:
        search_keys = search_criteria_dict.get(main_key)
        search_item = search_keys.items() 
         
        #initialize values
        search_field = ' '
        search_value = ' '
        search_action = ' '
        format_error = False
        #loop thru the matched key/values pairs in order to build search field, value and operand
        for y in search_item:
            if y[0] == 'field_name':
                search_field = y[1]
            elif y[0] == 'field_value':
                search_value = y[1] 
            elif y[0] == 'field_action':
                if y[1] in ('<','>','=','<=','>='):
                    search_action = y[1]
                else:
                    format_error = True
                    print('invalid field action in search criteria, search bypassed', search_item)
            else: 
                format_error = True
        #print('main_key', main_key)        
        if not format_error:
            # #build a dictionary from the CSV input file, this dictionary will be searched
            csv_data_file = load_csv(input_file_path)
            
            # call search_routine with the three search values and returns a dictionary with the information
            # found by the search funtion
            return_data, records_read, records_found = search_csv(search_field, search_value, search_action, csv_data_file)
            
            #write the results dictionary to a json file
            output_name = "data/search_results_" + str(main_key) + ".json" 
            print(f'The results for the requested search {main_key} is saved here {output_name}.')
            with open(output_name, "w") as outfile:
                json.dump(return_data, outfile)
            #write log file  
            logfile_name = "data/search_logfile.json" 
            log_entry = log_it(records_read,records_found,main_key) 
            print(f'A log file entry for the requested search {main_key} has been added to {logfile_name}.')
            #append entry
            with open(logfile_name, "a+") as logfile:
                json.dump(log_entry, logfile)
        else:
            print('malformed search criteria, search bypassed', search_item)
        
           

In [9]:
#main code
#file path name - C:\Users\Robert Buell\DAT-129_robert\Data\CPI.csv
input_file_path = path_names('data')
print('CSV input file path name ', input_file_path)
#file path name - C:\Users\Robert Buell\DAT-129_robert\Data\search_criteria.json
search_file_path = path_names('search')
print('JSON file with search criteria ',  search_file_path)


#pass the two file path names to the function process_search.  
process_search(search_file_path, input_file_path)

Enter a path for the input file: 
C:\Users\Robert Buell\DAT-129_robert\Data\incomes.csv
CSV input file path name  C:\Users\Robert Buell\DAT-129_robert\Data\incomes.csv
Enter a path for the search file: 
C:\Users\Robert Buell\DAT-129_robert\Data\search_criteria.json
JSON file with search criteria  C:\Users\Robert Buell\DAT-129_robert\Data\search_criteria.json
The results for the requested search search_value_1 is saved here data/search_results_search_value_1.json.
A log file entry for the requested search search_value_1 has been added to data/search_logfile.json.
The results for the requested search search_value_2 is saved here data/search_results_search_value_2.json.
A log file entry for the requested search search_value_2 has been added to data/search_logfile.json.
The results for the requested search search_value_3 is saved here data/search_results_search_value_3.json.
A log file entry for the requested search search_value_3 has been added to data/search_logfile.json.
The results for